In [ ]:
pip install geopandas folium matplotlib mapclassify

In [ ]:
#import needed modules
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import geopandas as gpd
import webbrowser
import os
from statsmodels.formula.api import ols
from scipy import stats
from datetime import datetime
from sklearn import preprocessing, linear_model
from sklearn.model_selection import KFold
from sklearn.linear_model import LinearRegression
from matplotlib.pyplot import figure
from mpl_toolkits.axes_grid1 import make_axes_locatable
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#your file path may be different
data = pd.read_csv("/content/drive/MyDrive/Colab/NJ/NJData.csv")
education = pd.read_csv("/content/drive/MyDrive/Colab/NJ/NJEducation.csv")
racial = pd.read_csv("/content/drive/MyDrive/Colab/NJ/NJRacial.csv")
townships = gpd.read_file("/content/drive/MyDrive/Colab/NJ/tl_2021_34_cousub/tl_2021_34_cousub.shp") 
townships['GEOID']=townships['GEOID'].astype(int)
counties = gpd.read_file("/content/drive/MyDrive/Colab/NJ/tl_2010_34_county10/tl_2010_34_county10.shp") 


In [ ]:
def merge(shapefile, dataset, column):
  return shapefile.merge(dataset, on=column)

In [ ]:
withElectionData = merge(townships,data,"GEOID")
withTwoData = merge(withElectionData,education,"GEOID")
filled = merge(withTwoData,racial,"GEOID")

In [ ]:
def electionInteractive(year, min, max):
  return filled.explore(column = year, tooltip = ["NAMELSAD","Pres16","Pres20","Swing"], cmap = "RdBu", vmin = min, vmax = max, style_kwds = {"fillOpacity":1,"weight":1,"color":"Black"})

In [ ]:
def demographicInteractive(demographic):
  return filled.explore(column = demographic, tooltip = ["NAMELSAD",demographic], cmap = "Greys", vmin = 0, vmax = 1, style_kwds = {"fillOpacity":1,"weight":1,"color":"Black"})

In [ ]:
def predictionInteractive(year, min, max):
  return filled.explore(column = year, tooltip = ["NAMELSAD","Pres20","Predicted","Difference"], cmap = "RdBu", vmin = min, vmax = max, style_kwds = {"fillOpacity":1,"weight":1,"color":"Black"})

In [ ]:
def plot(xAxis,yAxis):
  plot = filled.plot.scatter(x = xAxis, y = yAxis, c = 'DarkBlue')
  m, b = np.polyfit(filled[xAxis], filled[yAxis], 1)
  plt.plot(filled[xAxis], m*filled[xAxis]+b)

In [ ]:
def getDemo(i,type):
  return filled._get_value(i, type)

In [ ]:
def regression(xAxis,yAxis):
  model = ols(xAxis + " ~ " + yAxis, data=filled).fit()
  return model.summary()

In [ ]:
X = filled[['White', 'Black', 'Asian', 'Hispanic', 'Education']]
y = filled['Pres20']
regr = linear_model.LinearRegression()
regr.fit(X.values, y)

LinearRegression()

In [ ]:
Predicted = []
for i in range(len(filled)):
  Predicted.append(regr.predict([[getDemo(i,"White"),getDemo(i,"Black"),getDemo(i,"Asian"),getDemo(i,"Hispanic"),getDemo(i,"Education")]])[0])
filled['Predicted'] = Predicted
filled['Difference'] = filled['Predicted'] - filled['Pres20']

In [ ]:
electionMap = electionInteractive("Pres20",-0.8,0.8)
electionMap

In [ ]:
demographicMap = demographicInteractive("Education")
demographicMap

In [ ]:
predictedMap = predictionInteractive("Difference",-0.8,0.8)
predictedMap

In [ ]:
modelSummary = regression("White","Swing")
modelSummary

In [ ]:
plot("Asian","Difference")